In [1]:
import torch
from transformers import BertModel, BertTokenizer
import pandas as pd
import numpy as np
import pickle
from datetime import datetime


In [11]:
# 设置文件路径和参数
file_path = 'entity_description.txt'
batch_size = 16
max_length = 512


In [12]:
# 加载BERT模型和tokenizer
model_name = 'bert-base-uncased'
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# 读取文本文件
data = pd.read_csv(file_path, sep='\t', header=None, usecols=[1])
texts = data[1].tolist()


In [4]:
# 创建特征提取函数
def extract_features(texts):
    features = []
    num_batches = len(texts) // batch_size

    for i in range(num_batches):
        batch_texts = texts[i * batch_size: (i + 1) * batch_size]
        encoded_inputs = tokenizer(batch_texts, padding='longest', truncation=True, max_length=max_length, return_tensors='pt')
        with torch.no_grad():
            outputs = model(**encoded_inputs)
            features.append(outputs.last_hidden_state[:, 0, :].numpy())

        # 打印进度
        if (i + 1) % 10 == 0:
            timestamp = datetime.now().strftime("[%H:%M:%S]")
            completed_rows = (i + 1) * batch_size
            total_rows = num_batches * batch_size
            print(f"{timestamp} 已完成 {i + 1} 批次，{completed_rows}/{total_rows} 行")

    print("特征提取完成")
    return np.concatenate(features)


In [13]:
# 提取特征
text_features = extract_features(texts)


[18:28:42] 已完成 10 批次，160/12832 行
[18:29:23] 已完成 20 批次，320/12832 行
[18:30:07] 已完成 30 批次，480/12832 行
[18:30:52] 已完成 40 批次，640/12832 行
[18:31:37] 已完成 50 批次，800/12832 行
[18:32:22] 已完成 60 批次，960/12832 行
[18:33:06] 已完成 70 批次，1120/12832 行
[18:33:53] 已完成 80 批次，1280/12832 行
[18:34:37] 已完成 90 批次，1440/12832 行
[18:35:21] 已完成 100 批次，1600/12832 行
[18:36:06] 已完成 110 批次，1760/12832 行
[18:36:53] 已完成 120 批次，1920/12832 行
[18:37:40] 已完成 130 批次，2080/12832 行
[18:38:22] 已完成 140 批次，2240/12832 行
[18:39:06] 已完成 150 批次，2400/12832 行
[18:39:52] 已完成 160 批次，2560/12832 行
[18:40:38] 已完成 170 批次，2720/12832 行
[18:41:24] 已完成 180 批次，2880/12832 行
[18:42:10] 已完成 190 批次，3040/12832 行
[18:42:53] 已完成 200 批次，3200/12832 行
[18:43:39] 已完成 210 批次，3360/12832 行
[18:44:24] 已完成 220 批次，3520/12832 行
[18:45:06] 已完成 230 批次，3680/12832 行
[18:45:51] 已完成 240 批次，3840/12832 行
[18:46:35] 已完成 250 批次，4000/12832 行
[18:47:19] 已完成 260 批次，4160/12832 行
[18:48:03] 已完成 270 批次，4320/12832 行
[18:48:47] 已完成 280 批次，4480/12832 行
[18:49:32] 已完成 290 批次，4640/12832 行


In [14]:
# 保存特征为pickle文件
pickle.dump(text_features, open("text_features.pkl", "wb"))
